In [1]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np
import threading

In [2]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1  # 1: ±4.096V 범위 (ZMPT, ACS712에 적절)

# 센서 채널 지정
voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

# 보정 파라미터 (초기값)
ZMPT_offset = 2.510
ZMPT_scale = 941.68

ACS712_offset = 2.552
ACS712_sensitivity = 0.0925

In [3]:
# 데이터 저장 함수 (스레드에서 실행됨)
def save_data_async(data_chunk, count):
    df = pd.DataFrame(data_chunk, columns=['timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)'])
    df.to_csv("thread_raw_normal_dataset.csv", mode='a', index=False, header=False)

    # 평균 로그 출력
    print(f"[{count}] Mean raw V: {df['v_raw'].mean():.3f}, Mean raw C: {df['c_raw'].mean():.3f}")
    print(f"     Mean Real Voltage(V): {df['voltage(V)'].mean():.2f}, Current(A): {df['current(A)'].mean():.2f}")
    print("-" * 60)

In [4]:
# 초기화
data = []
count = 0

print("🟢 실시간 센서 감지 시작... (Ctrl+C로 종료)")

try:
    while True:
        now = time.time()

        # 센서 원시 전압값
        v_raw = voltage_ch.voltage
        c_raw = current_ch.voltage

        # 보정 계산
        real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        real_current = (c_raw - ACS712_offset) / ACS712_sensitivity

        # 데이터 추가
        data.append([now, v_raw, c_raw, real_voltage, real_current])
        count += 1

        # 100개마다 저장
        if len(data) >= 100:
            # 데이터 복사 후 비동기 저장
            threading.Thread(target=save_data_async, args=(data.copy(), count)).start()
            data = []  # 메인 루프용 버퍼 비움

        time.sleep(0.001)  # 최소 1ms 간격 (라즈베리파이에서는 이 이상 권장)
        # time.sleep(0.001163)  # 860Hz 샘플링 주기


except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

🟢 실시간 센서 감지 시작... (Ctrl+C로 종료)
[100] Mean raw V: 2.553, Mean raw C: 2.508
     Mean Real Voltage(V): 40.12, Current(A): -0.47
------------------------------------------------------------
[200] Mean raw V: 2.556, Mean raw C: 2.508
     Mean Real Voltage(V): 43.73, Current(A): -0.48
------------------------------------------------------------
[300] Mean raw V: 2.560, Mean raw C: 2.508
     Mean Real Voltage(V): 46.70, Current(A): -0.48
------------------------------------------------------------
[400] Mean raw V: 2.558, Mean raw C: 2.508
     Mean Real Voltage(V): 44.99, Current(A): -0.48
------------------------------------------------------------
[500] Mean raw V: 2.554, Mean raw C: 2.507
     Mean Real Voltage(V): 41.82, Current(A): -0.48
------------------------------------------------------------
[600] Mean raw V: 2.555, Mean raw C: 2.508
     Mean Real Voltage(V): 42.25, Current(A): -0.47
------------------------------------------------------------
[700] Mean raw V: 2.554, Mean raw